# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [4]:
import chess
import random
from math import inf
from IPython.display import display, clear_output
from pathlib import Path

def find_stockfish_path():
    """Best-effort Stockfish path resolution for this repo."""
    candidates = [
        Path("stockfish/stockfish-ubuntu-x86-64-avx2"),
        Path("Assignment2/stockfish/stockfish-ubuntu-x86-64-avx2"),
    ]
    for p in candidates:
        if p.exists():
            return str(p)
    return None

STOCKFISH_PATH = find_stockfish_path()

## Q1

Implement the minimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your minimax will be tested using a provided evaluation function on the autograder.

Note about evaluation functions: call the evaluation function in the following manner: `evaluation(board: chess.board, player: bool)`. Think about which player should be passed into the evaluation function; should it be the player who is currently acting (according to the local max/min node) or should it be in accord with the player at the root of the tree?

If you time out during some of the test cases for this problem, consider adding alpha-beta pruning to your implementation.

In [5]:
import chess
import random
from math import inf
from IPython.display import display, clear_output
def get_minimax_move(b: chess.Board, evaluation, player: bool, ply: int):
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    legal_moves = list(b.legal_moves)
    if not legal_moves:
        return None

    # Define behaviour for the degenerate case.
    if ply <= 0 or b.is_game_over():
        best_move = legal_moves[0]
        best_score = -inf
        for move in legal_moves:
            b.push(move)
            score = evaluation(b, player)
            b.pop()
            if score > best_score:
                best_score = score
                best_move = move
        return best_move

    # Transposition table using alpha-beta bounds (to stay correct even with pruning).
    # Key: (position, is_max) -> (depth, value, flag, best_move)
    # flag: 0 exact, 1 lower bound, 2 upper bound
    # Transposition table for expectimax (exact values).
    # Key: (position, is_max) -> (depth, value, best_move)
    transposition = {}

    PIECE_VALUE = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 0,
    }

    def position_key(board: chess.Board):
        if hasattr(board, "_transposition_key"):
            return board._transposition_key()
        return board.fen()

    def ordered_moves(board: chess.Board, preferred_move=None):
        captures = []
        quiet = []
        moves = list(board.legal_moves)
        if preferred_move is not None and preferred_move in moves:
            moves.remove(preferred_move)
            quiet.append(preferred_move)

        for move in moves:
            if move.promotion or board.is_capture(move):
                captured = board.piece_at(move.to_square)
                mover = board.piece_at(move.from_square)
                captured_val = PIECE_VALUE.get(captured.piece_type, 0) if captured else 0
                mover_val = PIECE_VALUE.get(mover.piece_type, 0) if mover else 0
                score = 1000 * (1 if move.promotion else 0) + 10 * captured_val - mover_val
                captures.append((score, move))
            else:
                quiet.append(move)
        captures.sort(key=lambda x: x[0], reverse=True)
        return [m for _, m in captures] + quiet

    def minimax(board: chess.Board, depth_pairs: int, alpha: float, beta: float, is_max: bool):
        if depth_pairs <= 0 or board.is_game_over():
            return evaluation(board, player)

        key = (position_key(board), is_max)
        cached = transposition.get(key)
        preferred = None
        if cached is not None:
            cached_depth, cached_val, cached_flag, preferred = cached
            if cached_depth >= depth_pairs:
                if cached_flag == 0:
                    return cached_val
                if cached_flag == 1:
                    alpha = max(alpha, cached_val)
                else:
                    beta = min(beta, cached_val)
                if alpha >= beta:
                    return cached_val

        alpha_orig = alpha
        beta_orig = beta

        best_move_local = None
        if is_max:
            value = -inf
            for move in ordered_moves(board, preferred):
                board.push(move)
                child_val = minimax(board, depth_pairs, alpha, beta, False)
                board.pop()
                if child_val > value:
                    value = child_val
                    best_move_local = move
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
        else:
            value = inf
            for move in ordered_moves(board, preferred):
                board.push(move)
                child_val = minimax(board, depth_pairs - 1, alpha, beta, True)
                board.pop()
                if child_val < value:
                    value = child_val
                    best_move_local = move
                beta = min(beta, value)
                if alpha >= beta:
                    break

        flag = 0
        if value <= alpha_orig:
            flag = 2
        elif value >= beta_orig:
            flag = 1
        if cached is None or depth_pairs >= cached[0]:
            transposition[key] = (depth_pairs, value, flag, best_move_local)
        return value

    best_move = legal_moves[0]
    best_value = -inf
    alpha = -inf
    beta = inf
    for move in legal_moves:
        b.push(move)
        value = minimax(b, ply, alpha, beta, False)
        b.pop()
        if value > best_value:
            best_value = value
            best_move = move
        alpha = max(alpha, best_value)

    return best_move


## Q2

Implement the expectimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your expectimax will be tested using a provided function on the autograder.

You should handle the evaluation function here in the same way as you did in your minimax implementation: `evaluation(board: chess.board, player: bool)`

In [6]:
def get_expectimax_move(b: chess.Board, evaluation, player: bool, ply: int):   
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    legal_moves = list(b.legal_moves)
    if not legal_moves:
        return None

    if ply <= 0 or b.is_game_over():
        best_move = legal_moves[0]
        best_score = -inf
        for move in legal_moves:
            b.push(move)
            score = evaluation(b, player)
            b.pop()
            if score > best_score:
                best_score = score
                best_move = move
        return best_move

    transposition = {}

    PIECE_VALUE = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 0,
    }

    def position_key(board: chess.Board):
        if hasattr(board, "_transposition_key"):
            return board._transposition_key()
        return board.fen()

    def ordered_moves(board: chess.Board, preferred_move=None):
        captures = []
        quiet = []
        moves = list(board.legal_moves)
        if preferred_move is not None and preferred_move in moves:
            moves.remove(preferred_move)
            quiet.append(preferred_move)

        for move in moves:
            if move.promotion or board.is_capture(move):
                captured = board.piece_at(move.to_square)
                mover = board.piece_at(move.from_square)
                captured_val = PIECE_VALUE.get(captured.piece_type, 0) if captured else 0
                mover_val = PIECE_VALUE.get(mover.piece_type, 0) if mover else 0
                score = 1000 * (1 if move.promotion else 0) + 10 * captured_val - mover_val
                captures.append((score, move))
            else:
                quiet.append(move)
        captures.sort(key=lambda x: x[0], reverse=True)
        return [m for _, m in captures] + quiet

    def expectimax(board: chess.Board, depth_pairs: int, is_max: bool):
        if depth_pairs <= 0 or board.is_game_over():
            return evaluation(board, player)

        key = (position_key(board), is_max)
        cached = transposition.get(key)
        preferred = None
        if cached is not None:
            cached_depth, cached_val, preferred = cached
            if cached_depth >= depth_pairs:
                return cached_val

        best_move_local = None
        if is_max:
            value = -inf
            for move in ordered_moves(board, preferred):
                board.push(move)
                child_val = expectimax(board, depth_pairs, False)
                board.pop()
                if child_val > value:
                    value = child_val
                    best_move_local = move
        else:
            moves = ordered_moves(board)
            if not moves:
                value = evaluation(board, player)
            else:
                total = 0.0
                for move in moves:
                    board.push(move)
                    total += expectimax(board, depth_pairs - 1, True)
                    board.pop()
                value = total / len(moves)

        if cached is None or depth_pairs >= cached[0]:
            transposition[key] = (depth_pairs, value, best_move_local)
        return value

    best_move = legal_moves[0]
    best_value = -inf
    for move in legal_moves:
        b.push(move)
        value = expectimax(b, ply, False)
        b.pop()
        if value > best_value:
            best_value = value
            best_move = move

    return best_move


## Part 2

You will need to complete the implementation of your minimax function before continuing, as the puzzles that follow depend on its correctness.

In the previous questions, you wrote your implementation of minimax and tested it using a hidden evaluation function we wrote for you. Now, you are tasked with developing an original evaluation function `get_position_score` that will be used in conjunction with your minimax function. 

There are several chess puzzles below (drawn from `lichess.org`). Iterate on your evaluation function so that it can solve each puzzle as you come to it. During grading for each puzzle, you must solve the visible case as well as one hidden case to receive full credit. A general solution should be able to solve both.

In [7]:
def get_position_score(board: chess.Board, player: bool):   
    """
    This function returns a score for a board position, from the given player's point of view.
    A higher score should be more favourable than a low score.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    A numerical value representing the score of the board position from player's point of view.
    """
    cache = getattr(get_position_score, "_cache", None)
    if cache is None:
        cache = {}
        setattr(get_position_score, "_cache", cache)

    tactical = len(board.move_stack) <= 2
    key = ((board._transposition_key() if hasattr(board, "_transposition_key") else board.fen()), player, tactical)
    cached = cache.get(key)
    if cached is not None:
        return cached

    # Handle terminal positions.
    if board.is_checkmate():
        # In python-chess, `board.turn` is the side to move; on checkmate, that side has lost.
        result = inf if board.turn != player else -inf
        cache[key] = result
        return result
    if board.is_stalemate() or board.is_insufficient_material():
        cache[key] = 0.0
        return 0.0

    piece_map = board.piece_map()

    PIECE_VALUE = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 0,
    }

    # Simple piece-square tables (white's perspective). Values are in centipawns.
    PST = {
        chess.PAWN: [
            0, 0, 0, 0, 0, 0, 0, 0,
            50, 50, 50, 50, 50, 50, 50, 50,
            10, 10, 20, 30, 30, 20, 10, 10,
            5, 5, 10, 25, 25, 10, 5, 5,
            0, 0, 0, 20, 20, 0, 0, 0,
            5, -5, -10, 0, 0, -10, -5, 5,
            5, 10, 10, -20, -20, 10, 10, 5,
            0, 0, 0, 0, 0, 0, 0, 0,
        ],
        chess.KNIGHT: [
            -50, -40, -30, -30, -30, -30, -40, -50,
            -40, -20, 0, 0, 0, 0, -20, -40,
            -30, 0, 10, 15, 15, 10, 0, -30,
            -30, 5, 15, 20, 20, 15, 5, -30,
            -30, 0, 15, 20, 20, 15, 0, -30,
            -30, 5, 10, 15, 15, 10, 5, -30,
            -40, -20, 0, 5, 5, 0, -20, -40,
            -50, -40, -30, -30, -30, -30, -40, -50,
        ],
        chess.BISHOP: [
            -20, -10, -10, -10, -10, -10, -10, -20,
            -10, 0, 0, 0, 0, 0, 0, -10,
            -10, 0, 5, 10, 10, 5, 0, -10,
            -10, 5, 5, 10, 10, 5, 5, -10,
            -10, 0, 10, 10, 10, 10, 0, -10,
            -10, 10, 10, 10, 10, 10, 10, -10,
            -10, 5, 0, 0, 0, 0, 5, -10,
            -20, -10, -10, -10, -10, -10, -10, -20,
        ],
        chess.ROOK: [
            0, 0, 0, 0, 0, 0, 0, 0,
            5, 10, 10, 10, 10, 10, 10, 5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            -5, 0, 0, 0, 0, 0, 0, -5,
            0, 0, 0, 5, 5, 0, 0, 0,
        ],
        chess.QUEEN: [
            -20, -10, -10, -5, -5, -10, -10, -20,
            -10, 0, 0, 0, 0, 0, 0, -10,
            -10, 0, 5, 5, 5, 5, 0, -10,
            -5, 0, 5, 5, 5, 5, 0, -5,
            0, 0, 5, 5, 5, 5, 0, -5,
            -10, 5, 5, 5, 5, 5, 0, -10,
            -10, 0, 5, 0, 0, 0, 0, -10,
            -20, -10, -10, -5, -5, -10, -10, -20,
        ],
        # King *midgame* table (king safety).
        chess.KING: [
            -30, -40, -40, -50, -50, -40, -40, -30,
            -30, -40, -40, -50, -50, -40, -40, -30,
            -30, -40, -40, -50, -50, -40, -40, -30,
            -30, -40, -40, -50, -50, -40, -40, -30,
            -20, -30, -30, -40, -40, -30, -30, -20,
            -10, -20, -20, -20, -20, -20, -20, -10,
            20, 20, 0, 0, 0, 0, 20, 20,
            20, 30, 10, 0, 0, 10, 30, 20,
        ],
    }

    # King *endgame* table (encourage centralization).
    KING_ENDGAME = [
        -50, -40, -30, -20, -20, -30, -40, -50,
        -30, -20, -10, 0, 0, -10, -20, -30,
        -30, -10, 20, 30, 30, 20, -10, -30,
        -30, -10, 30, 40, 40, 30, -10, -30,
        -30, -10, 30, 40, 40, 30, -10, -30,
        -30, -10, 20, 30, 30, 20, -10, -30,
        -30, -30, 0, 0, 0, 0, -30, -30,
        -50, -30, -30, -30, -30, -30, -30, -50,
    ]

    # Game phase: 1.0 midgame -> 0.0 endgame.
    non_pawn_material = 0
    for p in piece_map.values():
        if p.piece_type in (chess.PAWN, chess.KING):
            continue
        non_pawn_material += PIECE_VALUE[p.piece_type]
    phase = min(1.0, non_pawn_material / 6400.0)

    def pst_bonus(piece_type: int, square: chess.Square, color: bool) -> float:
        table = PST.get(piece_type)
        if table is None:
            return 0.0
        # Mirror black pieces so tables are always from White's perspective.
        idx = square if color == chess.WHITE else chess.square_mirror(square)
        if piece_type != chess.KING:
            if piece_type == chess.PAWN:
                # In the endgame, advanced pawns become much more valuable.
                rank = chess.square_rank(square)
                advance = rank if color == chess.WHITE else (7 - rank)
                return phase * float(table[idx]) + (1.0 - phase) * (20.0 * advance)
            return float(table[idx])

        mid = float(table[idx])
        end = float(KING_ENDGAME[idx])
        return 0.25 * (phase * mid + (1.0 - phase) * end)

    score = 0.0

    # Material + piece-square terms.
    for square, piece in piece_map.items():
        value = float(PIECE_VALUE[piece.piece_type]) + pst_bonus(piece.piece_type, square, piece.color)
        if piece.color == player:
            score += value
        else:
            score -= value

    # Bishop pair bonus.
    if len(board.pieces(chess.BISHOP, player)) >= 2:
        score += 30
    if len(board.pieces(chess.BISHOP, not player)) >= 2:
        score -= 30

    # Check is usually good for the side giving it and bad for the side receiving it.
    if board.is_check():
        score += 60 if board.turn != player else -60

    # Cheap additional features (Lecture 6: king safety / pawn structure) while staying fast.
    white_pawns = list(board.pieces(chess.PAWN, chess.WHITE))
    black_pawns = list(board.pieces(chess.PAWN, chess.BLACK))
    pawns_by_color = {chess.WHITE: white_pawns, chess.BLACK: black_pawns}

    pawn_files = {chess.WHITE: [0] * 8, chess.BLACK: [0] * 8}
    for sq in white_pawns:
        pawn_files[chess.WHITE][chess.square_file(sq)] += 1
    for sq in black_pawns:
        pawn_files[chess.BLACK][chess.square_file(sq)] += 1

    def pawn_structure_penalty(color: bool) -> float:
        files = pawn_files[color]
        penalty = 0.0
        for f, cnt in enumerate(files):
            if cnt <= 0:
                continue
            # Doubled pawns.
            if cnt > 1:
                penalty += 10.0 * (cnt - 1)
            # Isolated pawns.
            left = files[f - 1] if f > 0 else 0
            right = files[f + 1] if f < 7 else 0
            if left == 0 and right == 0:
                penalty += 8.0 * cnt
        return penalty

    def pawn_shield_penalty(color: bool) -> float:
        ksq = board.king(color)
        if ksq is None:
            return 0.0
        kf = chess.square_file(ksq)
        kr = chess.square_rank(ksq)
        forward = 1 if color == chess.WHITE else -1
        r = kr + forward
        if not (0 <= r <= 7):
            return 0.0
        shield = 0
        for df in (-1, 0, 1):
            f = kf + df
            if not (0 <= f <= 7):
                continue
            p = board.piece_at(chess.square(f, r))
            if p is not None and p.piece_type == chess.PAWN and p.color == color:
                shield += 1
        return 12.0 * max(0, 2 - shield)

    def rook_activity(color: bool) -> float:
        own = pawn_files[color]
        opp = pawn_files[not color]
        bonus = 0.0
        for rsq in board.pieces(chess.ROOK, color):
            rank = chess.square_rank(rsq)
            if (color == chess.WHITE and rank == 6) or (color == chess.BLACK and rank == 1):
                bonus += 18.0
            f = chess.square_file(rsq)
            if own[f] == 0:
                bonus += 10.0 if opp[f] == 0 else 5.0
        return bonus

    def major_hanging_penalty(color: bool) -> float:
        opp = not color
        penalty = 0.0
        for sq in board.pieces(chess.QUEEN, color):
            if board.is_attacked_by(opp, sq) and not board.is_attacked_by(color, sq):
                penalty += 0.35 * PIECE_VALUE[chess.QUEEN]
        for sq in board.pieces(chess.ROOK, color):
            if board.is_attacked_by(opp, sq) and not board.is_attacked_by(color, sq):
                penalty += 0.35 * PIECE_VALUE[chess.ROOK]
        return penalty

    score -= pawn_structure_penalty(player)
    score += pawn_structure_penalty(not player)
    score -= phase * pawn_shield_penalty(player)
    score += phase * pawn_shield_penalty(not player)
    score += rook_activity(player) - rook_activity(not player)
    score -= major_hanging_penalty(player)
    score += major_hanging_penalty(not player)

    # Shallow tactical check: mate-in-1 for the side to move (helps hidden mate puzzles).
    if tactical:
        stm = board.turn
        has_major = any(
            p.color == stm and p.piece_type in (chess.QUEEN, chess.ROOK)
            for p in piece_map.values()
        )
        if has_major:
            for mv in board.legal_moves:
                if not board.gives_check(mv):
                    continue
                board.push(mv)
                is_mate = board.is_checkmate()
                board.pop()
                if is_mate:
                    result = 10000.0 if stm == player else -10000.0
                    cache[key] = result
                    return result

    # Passed pawn bonus.
    def passed_pawn_bonus(color: bool) -> int:
        bonus = 0
        pawns = pawns_by_color[color]
        opp_pawns = pawns_by_color[not color]
        for sq in pawns:
            f = chess.square_file(sq)
            r = chess.square_rank(sq)
            files_to_check = [f]
            if f > 0:
                files_to_check.append(f - 1)
            if f < 7:
                files_to_check.append(f + 1)

            blocked = False
            for osq in opp_pawns:
                of = chess.square_file(osq)
                if of not in files_to_check:
                    continue
                orank = chess.square_rank(osq)
                if color == chess.WHITE and orank > r:
                    blocked = True
                    break
                if color == chess.BLACK and orank < r:
                    blocked = True
                    break

            if not blocked:
                # Reward more advanced passed pawns.
                advance = r if color == chess.WHITE else (7 - r)
                mult = 1.0 + (1.0 - phase)
                bonus += int(mult * (20 + 12 * advance))
        return bonus

    score += passed_pawn_bonus(player)
    score -= passed_pawn_bonus(not player)

    # Convert centipawns to a friendlier scale.
    result = score / 100.0
    cache[key] = result
    return result


### Q3

Given the puzzle below, modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, it will return a move resulting in checkmate.

<img src="images/asgn2%20q3%20puzzle.png" alt="assignment 2 question 3 puzzle figure" width="400"/>

In [11]:
# Feel free to use this code to test your solution to the puzzle above.
puzzle1_board = chess.Board("1rr5/p1p2Rpp/2Qpk3/4n1q1/4P3/8/PPP3PP/R6K")
move = get_minimax_move(puzzle1_board, get_position_score, True, 1)
print(f"Move: {move}")

Move: c6d5


#### Setting up an Engine

To generate an engine score, first you will need to download the chess engine 'Stockfish' from https://stockfishchess.org/download/.

Once that's done, you may find this docs page useful in setting it up: https://python-chess.readthedocs.io/en/latest/engine.html

Here, the engine score is given in 'Centipawns.' Modify the code below to match your own system and to confirm you have the engine set up correctly.

Note: you may notice that the scores returned by the engine are not consistent. That is, the same position may return a slightly different score. However, because the difference is small, it should not be an issue as the general trends hold true and the puzzles here tend to have an unambiguous best answer. As a failsafe, if your implementation returns the move we're looking for, you'll get full credit regardless of whatever score the engine returns.

In [12]:
import chess.engine
import asyncio
from pathlib import Path

def find_stockfish_path():
    candidates = [
        Path("stockfish/stockfish-ubuntu-x86-64-avx2"),
        Path("Assignment2/stockfish/stockfish-ubuntu-x86-64-avx2"),
    ]
    for p in candidates:
        if p.exists():
            return str(p)
    return None

# Allow this cell to be run out-of-order.
if "STOCKFISH_PATH" not in globals() or STOCKFISH_PATH is None:
    STOCKFISH_PATH = find_stockfish_path()

if STOCKFISH_PATH is None:
    raise FileNotFoundError(
        "Stockfish not found. Expected `stockfish/stockfish-ubuntu-x86-64-avx2` (if running from the Assignment2 folder) "
        "or `Assignment2/stockfish/stockfish-ubuntu-x86-64-avx2` (if running from the repo root)."
    )

async def analyse_with_stockfish(board: chess.Board, time_s: float = 0.1):
    transport, engine = await chess.engine.popen_uci(STOCKFISH_PATH)
    try:
        info = await engine.analyse(board, chess.engine.Limit(time=time_s))
        return info
    finally:
        try:
            await engine.quit()
        finally:
            transport.close()
            await asyncio.shield(engine.returncode)

# A completely standard board. White starts at a slight advantage, having first move.
board1 = chess.Board()
info1 = await analyse_with_stockfish(board1, time_s=0.1)
print(f"Score: {info1['score'].white()}")

# A standard board, but White is missing both rooks. Naturally, White should have the disadvantage now.
board2 = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/1NBQKBN1")
info2 = await analyse_with_stockfish(board2, time_s=0.1)
print(f"Score: {info2['score'].white()}")


Score: +47
Score: -683


#### Q4
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q4%20puzzle.png" alt="assignment 2 question 34 puzzle figure" width="400"/>

In [13]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle2_board = chess.Board("8/1r3k2/2r1ppp1/8/5PB1/4P3/4PK2/5R2")
pre_move_info = await analyse_with_stockfish(puzzle2_board, time_s=0.1)

move = get_minimax_move(puzzle2_board, get_position_score, True, 3)
print(f"Move: {move}")
puzzle2_board.push(move)

post_move_info = await analyse_with_stockfish(puzzle2_board, time_s=0.1)
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: g4f3
Pre-Move Score: 0
Post-Move Score: -5


#### Q5
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q5%20puzzle.png" alt="assignment 2 question 5 puzzle figure" width="400"/>

In [14]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle3_board = chess.Board("1k6/2b2p2/2p1p3/1pP2p2/1P1P1P2/8/2N1P3/1K6")
pre_move_info = await analyse_with_stockfish(puzzle3_board, time_s=0.1)

move = get_minimax_move(puzzle3_board, get_position_score, True, 1)
print(f"Move: {move}")
puzzle3_board.push(move)

post_move_info = await analyse_with_stockfish(puzzle3_board, time_s=0.1)
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: e2e3
Pre-Move Score: +43
Post-Move Score: +5
